In [167]:
#https://github.com/mariovitale1979/sync-plugin.git
#https://github.com/corvus-albus/moodle-local_wsmanagesections/

import requests
import json

def rest_api_parameters(in_args, prefix='', out_dict=None):
    if out_dict==None:
        out_dict = {}
    if not type(in_args) in (list,dict):
        out_dict[prefix] = in_args
        return out_dict
    if prefix == '':
        prefix = prefix + '{0}'
    else:
        prefix = prefix + '[{0}]'
    if type(in_args)==list:
        for idx, item in enumerate(in_args):
            rest_api_parameters(item, prefix.format(idx), out_dict)
    elif type(in_args)==dict:
        for key, item in in_args.items():
            rest_api_parameters(item, prefix.format(key), out_dict)
    return out_dict

class MoodleCourseCreator:
    def __init__(self, domain, token):
        """
        Initialize with Moodle domain and Web Services token
        """
        self.domain = domain.rstrip('/')
        self.token = token
        self.base_url = f"{self.domain}/webservice/rest/server.php"

    def call_moodle_api(self, wsfunction, params=None):
        """Make a request to the Moodle API"""
        if params is None:
            params = {}
            
        parameters = rest_api_parameters(params)
        parameters.update({
            'wstoken': self.token,
            'moodlewsrestformat': 'json',
            'wsfunction': wsfunction
        })
        
        response = requests.post(self.base_url, data=parameters)

        print(f"API Call URL: {response.url}")
        print(f"Raw response: {response.text}")
        
        if response.status_code != 200:
            raise Exception(f"API call failed with status code: {response.status_code}")
            
        try:
            result = response.json()
            if isinstance(result, dict) and 'exception' in result:
                raise Exception(f"Moodle API error: {result['message']}\nDebug info: {result.get('debuginfo', '')}")
            return result
        except json.JSONDecodeError as e:
            print(f"Raw response: {response.text}")
            raise Exception(f"Failed to decode JSON response: {str(e)}")

    def create_course(self, full_name, short_name, category_id=1, num_sections=0):
        """Create a new course with specified number of sections"""
        params = {
            'courses': [{
                'fullname': full_name,
                'shortname': short_name,
                'categoryid': category_id,
                'format': 'topics',
                'numsections': num_sections
            }]
        }
        
        result = self.call_moodle_api('core_course_create_courses', params)
        if isinstance(result, list) and result:
            return result[0].get('id')
        return None

    def add_section_to_course(self, course_id, title, position=0):
        params = {
            "courseid": course_id,
            "sectionname": title,
            "sectionnum": position            
        }
        return self.call_moodle_api('local_course_add_new_section', params)
        

    def get_course_sections(self, course_id):
        """Get the sections of a course"""
        return self.call_moodle_api('local_wsmanagesections_get_sections', {'courseid': course_id})


    def add_page_to_section(self, course_id, section_number, title, content):
        params = {
            "courseid": course_id, 
            "sectionnum": section_number,
            "urlname": title,
            "content": content, 
            "visible": 1
        }
        return self.call_moodle_api('local_course_add_new_course_module_page', params)

    def update_page_content(self, page_id, content):
        params = {
            "cmid": page_id,
            "content": content, 
        }
        return self.call_moodle_api('local_course_update_course_module_page', params)


# Example usage
MOODLE_URL = "http://localhost:8081/"
TOKEN = "3e532616b29a4a6e322f7c05ee407c79"

creator = MoodleCourseCreator(MOODLE_URL, TOKEN)
course_num = 7

In [168]:
course_num = course_num+1
course_id = creator.create_course(
    full_name=f"mycourse-{course_num}",
    short_name=f"mycourse-{course_num}",
    category_id=1,
    num_sections=0
)


API Call URL: http://localhost:8081/webservice/rest/server.php
Raw response: [{"id":23,"shortname":"mycourse-8"}]


In [173]:
section_pos = 0
section = creator.add_section_to_course(course_id, "dddd section", section_pos)
section

API Call URL: http://localhost:8081/webservice/rest/server.php
Raw response: {"message":"Successful"}


{'message': 'Successful'}

In [175]:
page = creator.add_page_to_section(course_id , 1, "ma page de cours", "<h1>titre du chapitre....</h1>")
page

API Call URL: http://localhost:8081/webservice/rest/server.php
Raw response: {"message":"Successful","id":"52"}


{'message': 'Successful', 'id': '52'}

In [177]:
updatedPage = creator.update_page_content(page.get('id'), "<h1>chapter title</h1>")

API Call URL: http://localhost:8081/webservice/rest/server.php
Raw response: {"message":"Successful","id":"52"}


In [178]:
final_sections = creator.get_course_sections(course_id)
final_sections

API Call URL: http://localhost:8081/webservice/rest/server.php
Raw response: [{"sectionnum":0,"id":57,"name":"General","summary":"","summaryformat":1,"visible":1,"uservisible":true,"availability":null,"sequence":"50,51","courseformat":"topics","sectionformatoptions":[]},{"sectionnum":1,"id":58,"name":"bbb section","summary":"","summaryformat":1,"visible":1,"uservisible":true,"availability":null,"sequence":"52","courseformat":"topics","sectionformatoptions":[]},{"sectionnum":2,"id":59,"name":"aaa section","summary":"","summaryformat":1,"visible":1,"uservisible":true,"availability":null,"sequence":"","courseformat":"topics","sectionformatoptions":[]},{"sectionnum":3,"id":60,"name":"ccc section","summary":"","summaryformat":1,"visible":1,"uservisible":true,"availability":null,"sequence":"","courseformat":"topics","sectionformatoptions":[]},{"sectionnum":4,"id":61,"name":"dddd section","summary":"","summaryformat":1,"visible":1,"uservisible":true,"availability":null,"sequence":"","coursefo

[{'sectionnum': 0,
  'id': 57,
  'name': 'General',
  'summary': '',
  'summaryformat': 1,
  'visible': 1,
  'uservisible': True,
  'availability': None,
  'sequence': '50,51',
  'courseformat': 'topics',
  'sectionformatoptions': []},
 {'sectionnum': 1,
  'id': 58,
  'name': 'bbb section',
  'summary': '',
  'summaryformat': 1,
  'visible': 1,
  'uservisible': True,
  'availability': None,
  'sequence': '52',
  'courseformat': 'topics',
  'sectionformatoptions': []},
 {'sectionnum': 2,
  'id': 59,
  'name': 'aaa section',
  'summary': '',
  'summaryformat': 1,
  'visible': 1,
  'uservisible': True,
  'availability': None,
  'sequence': '',
  'courseformat': 'topics',
  'sectionformatoptions': []},
 {'sectionnum': 3,
  'id': 60,
  'name': 'ccc section',
  'summary': '',
  'summaryformat': 1,
  'visible': 1,
  'uservisible': True,
  'availability': None,
  'sequence': '',
  'courseformat': 'topics',
  'sectionformatoptions': []},
 {'sectionnum': 4,
  'id': 61,
  'name': 'dddd section',
